In [1]:
import pandas as pd
import geopandas as gpd
import os, glob

In [2]:
run_dir = 'M:\\Application\\Model One\\RTP2021\\Blueprint\\VMT per capita or worker'
os.chdir(run_dir)

In [3]:
taz = gpd.read_file(r'M:\Data\GIS layers\Travel_Analysis_Zones_(TAZ1454)\Travel Analysis Zones.shp')
print('Read {} rows of TAZ1454 shapefile'.format(taz.shape[0]))
print(list(taz))
taz = taz[['SUPERD','TAZ1454']]
display(taz.head())

Read 1454 rows of TAZ1454 shapefile
['SUPERD', 'TAZ1454', 'Shape__Are', 'Shape__Len', 'geometry']


,SUPERD,TAZ1454
0,18,939
1,18,974
2,18,914
3,18,920
4,18,921


In [4]:
SD_name = pd.read_csv(r'M:\Data\GIS layers\TM1_taz\superdistrict_names.csv')
print('Read {} rows of superdistrict ID-name lookup'.format(SD_name.shape[0]))
display(SD_name.head())

taz_county = pd.read_csv(r'C:\Users\{}\Documents\GitHub\petrale\zones\taz1454\taz1454_names.csv'.format(os.getenv('USERNAME')),
                        usecols = ['taz_key', 'county'])
print('Read {} rows of TAZ-county lookup'.format(taz_county.shape[0]))

county_dict = {'sfr': 'San Francisco',
               'smt': 'San Mateo',
               'scl': 'Santa Clara',
               'ala': 'Alameda',
               'cnc': 'Contra Costa',
               'sol': 'Solano',
               'nap': 'Napa',
               'son': 'Sonoma',
               'mar': 'Marin'}
taz_county['county'] = taz_county['county'].map(county_dict)
display(taz_county.head())

Read 34 rows of superdistrict ID-name lookup


,SUPERD,SD_NAME
0,1,Greater Downtown San Francisco
1,2,San Francisco Richmond District
2,3,San Francisco Mission District
3,4,San Francisco Sunset District
4,5,Daly City and San Bruno


Read 1454 rows of TAZ-county lookup


,taz_key,county
0,1,San Francisco
1,2,San Francisco
2,3,San Francisco
3,4,San Francisco
4,5,San Francisco


In [5]:
taz = taz.merge(taz_county,
                left_on='TAZ1454',
                right_on='taz_key',
                how='inner').merge(SD_name,
                                   on='SUPERD',
                                   how='left')
taz.drop(columns=['taz_key'], inplace=True)

In [6]:
taz.to_csv(os.path.join(run_dir, 'Tableau', 'taz_index.csv'), index=False)

### home vmt

In [7]:
def vmt_percapita_cat(x):
    if x > 20:
        return 'More than 20'
    elif x > 15:
        return '15 to 20'
    elif x > 10:
        return '10 to 15'
    elif x > 5:
        return '5 to 10'
    else:
        return 'Less than 5'

def vmt_perworker_cat(x):
    if x > 30:
        return 'More than 30'
    elif x > 25:
        return '25 to 30'
    elif x > 20:
        return '20 to 25'
    elif x > 15:
        return '15 to 20'
    else:
        return 'Less than 15'

In [8]:
vmt_by_TAZ = pd.DataFrame(columns=['TAZ1454',
                                   'total_vmt_resident', 'total_pop_resident',
                                   'vmtpercapita', 'vmtpercapitaCat',
                                   'total_vmt_worker', 'total_pop_worker',
                                   'vmtperworker', 'vmtperworkerCat', 'type', 'scen', 'year', 'run'])

vmt_by_SD = pd.DataFrame(columns=['SUPERD', 'SD_NAME',
                                  'total_vmt_resident', 'total_pop_resident',
                                  'vmtpercapita', 'vmtpercapitaCat',
                                  'total_vmt_worker', 'total_pop_worker',
                                  'vmtperworker', 'vmtperworkerCat', 'type', 'scen', 'year', 'run'])

vmt_by_COUNTY = pd.DataFrame(columns=['county',
                                      'total_vmt_resident', 'total_pop_resident',
                                      'vmtpercapita', 'vmtpercapitaCat',
                                      'total_vmt_worker', 'total_pop_worker',
                                      'vmtperworker', 'vmtperworkerCat', 'type', 'scen', 'year', 'run'])


for file in list(glob.glob('*.csv')):
#     print(file)
    
    if 'Home' in file:
        df = pd.read_csv(file, usecols = ['taz', 'total_vmt', 'total_pop', 'vmtpercapita'])
        df.rename(columns = {'taz': 'TAZ1454',
                             'total_vmt': 'total_vmt_resident',
                             'total_pop': 'total_pop_resident'}, inplace=True)
    elif 'Work' in file:
        df = pd.read_csv(file, usecols = ['WorkLocation', 'total_vmt', 'total_pop', 'vmtperworker'])
        df.rename(columns = {'WorkLocation': 'TAZ1454',
                             'total_vmt': 'total_vmt_worker',
                             'total_pop': 'total_pop_worker'}, inplace=True)
        
    print('{} has {} rows of TAZ summary data'.format(file, df.shape[0]))
    
    df = df.merge(taz, on='TAZ1454', how='right')
#     display(df.head())

    # summarize by SD and County
    if 'Home' in file:
        for i in ['total_vmt_resident', 'total_pop_resident']:
            df[i] = df[i].fillna(0)

        df_SD = df.groupby(['SUPERD', 'SD_NAME'])['total_vmt_resident', 'total_pop_resident'].sum().reset_index()
        df_SD['vmtpercapita'] = df_SD['total_vmt_resident']/df_SD['total_pop_resident']
        
        df_county = df.groupby('county')['total_vmt_resident', 'total_pop_resident'].sum().reset_index()
        df_county['vmtpercapita'] = df_county['total_vmt_resident']/df_county['total_pop_resident']
        
    if 'Work' in file:
        for i in ['total_vmt_worker', 'total_pop_worker']:
            df[i] = df[i].fillna(0)
        
        df_SD = df.groupby(['SUPERD', 'SD_NAME'])['total_vmt_worker', 'total_pop_worker'].sum().reset_index()
        df_SD['vmtperworker'] = df_SD['total_vmt_worker']/df_SD['total_pop_worker']
        
        df_county = df.groupby('county')['total_vmt_worker', 'total_pop_worker'].sum().reset_index()
        df_county['vmtperworker'] = df_county['total_vmt_worker']/df_county['total_pop_worker']
        
       
    # add fields to represent output scenario, year, and run_id
    run_year = file.split('.')[0].split('_')[1]

    if 'TM152' in file:
        if file.split('.')[0].split('_')[4] == 'NoProject':
            scen = 'PBA50_NP'
            run_id = file.split('.')[0].split('_')[5]
        elif file.split('.')[0].split('_')[4] == 'Plus':
            scen = 'PBA50_Plus'
            run_id = file.split('.')[0].split('_')[5]
        elif file.split('.')[0].split('_')[4] == 'PlusCrossing':
            scen = 'PBA50_Plus'
            run_id = file.split('.')[0].split('_')[5]
        else:
            scen = 'PBA50_' + file.split('.')[0].split('_')[4]
            run_id = file.split('.')[0].split('_')[5]
    elif '06_694' in file:
        scen = 'PBA40'
        run_id = file.split('.')[0][10:]

    print('scen: {}, year: {}, run: {}'.format(scen, run_year, run_id))
    
    if 'Home' in file:
        df['type'] = 'resident'
        df_SD['type'] = 'resident'
        df_county['type'] = 'resident'
    elif 'Work' in file:
        df['type'] = 'worker'
        df_SD['type'] = 'worker'
        df_county['type'] = 'worker'
    
    df['scen'] = scen
    df_SD['scen'] = scen
    df_county['scen'] = scen
    
    df['year'] = run_year
    df_SD['year'] = run_year
    df_county['year'] = run_year
    
    df['run'] = run_id
    df_SD['run'] = run_id
    df_county['run'] = run_id

    if 'Home' in file:
        df['vmtpercapitaCat'] = df['vmtpercapita'].apply(lambda x: vmt_percapita_cat(x))
        df_SD['vmtpercapitaCat'] = df_SD['vmtpercapita'].apply(lambda x: vmt_percapita_cat(x))
        df_county['vmtpercapitaCat'] = df_county['vmtpercapita'].apply(lambda x: vmt_percapita_cat(x))
    elif 'Work' in file:    
        df['vmtperworkerCat'] = df['vmtperworker'].apply(lambda x: vmt_perworker_cat(x))
        df_SD['vmtperworkerCat'] = df_SD['vmtperworker'].apply(lambda x: vmt_perworker_cat(x))
        df_county['vmtperworkerCat'] = df_county['vmtperworker'].apply(lambda x: vmt_perworker_cat(x))
    
#     display(df)
#     display(df_SD)
#     display(df_county)

    vmt_by_TAZ = pd.concat([vmt_by_TAZ, df])
    vmt_by_SD = pd.concat([vmt_by_SD, df_SD])
    vmt_by_COUNTY = pd.concat([vmt_by_COUNTY, df_county])
    

Home_2025_TM152_FBP_Plus_22.csv has 1445 rows of TAZ summary data
scen: PBA50_Plus, year: 2025, run: 22
Home_2030_06_694.csv has 1441 rows of TAZ summary data
scen: PBA40, year: 2030, run: 06_694
Home_2030_TM152_FBP_Plus_23.csv has 1447 rows of TAZ summary data
scen: PBA50_Plus, year: 2030, run: 23
Home_2035_TM152_EIR_Alt1_06.csv has 1447 rows of TAZ summary data
scen: PBA50_Alt1, year: 2035, run: 06
Home_2035_TM152_EIR_Alt2_04.csv has 1449 rows of TAZ summary data
scen: PBA50_Alt2, year: 2035, run: 04


C:\Users\ywang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:44: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
C:\Users\ywang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:47: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


Home_2035_TM152_FBP_NoProject_24.csv has 1446 rows of TAZ summary data
scen: PBA50_NP, year: 2035, run: 24
Home_2035_TM152_FBP_Plus_24.csv has 1448 rows of TAZ summary data
scen: PBA50_Plus, year: 2035, run: 24
Home_2040_06_694_Amd1.csv has 1441 rows of TAZ summary data
scen: PBA40, year: 2040, run: 06_694_Amd1
Home_2040_TM152_FBP_Plus_24.csv has 1448 rows of TAZ summary data
scen: PBA50_Plus, year: 2040, run: 24
Home_2050_TM152_EIR_Alt1_06.csv has 1449 rows of TAZ summary data
scen: PBA50_Alt1, year: 2050, run: 06
Home_2050_TM152_EIR_Alt2_05.csv has 1448 rows of TAZ summary data
scen: PBA50_Alt2, year: 2050, run: 05
Home_2050_TM152_FBP_NoProject_24.csv has 1444 rows of TAZ summary data
scen: PBA50_NP, year: 2050, run: 24
Home_2050_TM152_FBP_PlusCrossing_24.csv has 1449 rows of TAZ summary data
scen: PBA50_Plus, year: 2050, run: 24
Work_2025_TM152_FBP_Plus_22.csv has 1453 rows of TAZ summary data
scen: PBA50_Plus, year: 2025, run: 22
Work_2030_06_694.csv has 1455 rows of TAZ summary da

C:\Users\ywang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:54: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
C:\Users\ywang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:57: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



scen: PBA40, year: 2030, run: 06_694
Work_2030_TM152_FBP_Plus_23.csv has 1453 rows of TAZ summary data
scen: PBA50_Plus, year: 2030, run: 23
Work_2035_TM152_EIR_Alt1_06.csv has 1452 rows of TAZ summary data
scen: PBA50_Alt1, year: 2035, run: 06
Work_2035_TM152_EIR_Alt2_04.csv has 1453 rows of TAZ summary data
scen: PBA50_Alt2, year: 2035, run: 04
Work_2035_TM152_FBP_NoProject_24.csv has 1452 rows of TAZ summary data
scen: PBA50_NP, year: 2035, run: 24
Work_2035_TM152_FBP_Plus_24.csv has 1453 rows of TAZ summary data
scen: PBA50_Plus, year: 2035, run: 24
Work_2040_06_694_Amd1.csv has 1455 rows of TAZ summary data
scen: PBA40, year: 2040, run: 06_694_Amd1
Work_2040_TM152_FBP_Plus_24.csv has 1453 rows of TAZ summary data
scen: PBA50_Plus, year: 2040, run: 24
Work_2050_TM152_EIR_Alt1_06.csv has 1453 rows of TAZ summary data
scen: PBA50_Alt1, year: 2050, run: 06
Work_2050_TM152_EIR_Alt2_05.csv has 1451 rows of TAZ summary data
scen: PBA50_Alt2, year: 2050, run: 05
Work_2050_TM152_FBP_NoPro

In [9]:
vmt_by_TAZ.to_csv(os.path.join(run_dir, 'Tableau', 'vmt_by_TAZ.csv'), index=False)
vmt_by_SD.to_csv(os.path.join(run_dir, 'Tableau', 'vmt_by_SD.csv'), index=False)
vmt_by_COUNTY.to_csv(os.path.join(run_dir, 'Tableau', 'vmt_by_COUNTY.csv'), index=False)